In [10]:
import os
import json
import base64
import requests
import pandas as pd
import tensorflow as tf

In [11]:
from pprint import PrettyPrinter

pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/movie-reviews-sentiment-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1680333859'}]}


In [12]:
# path
ENCODE_CSV_PATH = 'data/encode_data'

df = pd.read_csv(os.path.join(ENCODE_CSV_PATH, 'movie_reviews_sentiment.csv'))
review = df['review'][2]
review

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

In [15]:
def prepare_json(text):
    feature_spec = {
        'text': tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, 'utf-8')]))
    }

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{
        'examples': {
            'b64': base64.b64encode(example).decode()
        }
    }]

    return json.dumps({
        'signature_name': 'serving_default',
        'instances': result
    })

In [18]:
def make_predictions(text):
    json_ = prepare_json(text)
    endpoint = 'http://localhost:8080/v1/models/movie-reviews-sentiment-model:predict'

    response = requests.post(endpoint, data=json_)

    predict = response.json()['predictions'][0][0]
    return predict

In [19]:
predict = make_predictions(review)

print(f'{review}\n')
print(f'Movie reviews are positive sentiment ({predict})' if predict > 0.6 else f'Movie reviews are negative sentiment ({predict})')

KeyError: 'predictions'